<a href="https://www.kaggle.com/code/nikoladyulgerov/r-pido-y-furioso-el-precio-de-la-gasolina?scriptVersionId=228309011" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd 
import plotly.express as px
import plotly.io as pio

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Descripción general de los datos

In [3]:
df_gasolineras = pd.read_csv("/kaggle/input/precios-de-gasolineras-de-albacete/gasolineras_ab.csv")
df_precios = pd.read_csv("/kaggle/input/precios-de-gasolineras-de-albacete/precios_gasolineras.csv", parse_dates=['fecha'], date_format='%d/%m/%y' )

In [4]:
df_gasolineras.head()

,id_estacion,direccion,cod_postal,latitud,longitud,rotulo,horario,municipio
0,10765,"AVENIDA 1º DE MAYO, S/N",2001,38.985667,-1.868500,CARREFOUR,L-S: 08:00-22:00; D: 09:00-21:00,Albacete
1,12054,CALLE PRINCIPE DE ASTURIAS (POLÍGONO DE ROMICA...,2001,39.054694,-1.832000,BP ROMICA,L-D: 06:00-22:00,Albacete
2,13933,"CALLE FEDERICO GARCIA LORCA, 1",2001,39.000861,-1.849833,PLENOIL,L-D: 24H,Albacete
3,4369,"AVENIDA MENÉNDEZ PIDAL, 58",2005,39.003333,-1.864917,TAMOS,L-D: 24H,Albacete
4,5195,"CL PASEO DE LA CUBA, 15",2005,38.999722,-1.854556,REPSOL,L-D: 06:00-22:00,Albacete


In [5]:
df_precios.head()

,id_estacion,fecha,precio_gasoleo_a,precio_gasoleo_premium,precio_gasolina_95,precio_gasolina_98
0,10765,01/01/2022,1.399,1.399,1.519,1.639
1,12054,01/01/2022,1.389,1.389,1.479,1.621
2,13933,01/01/2022,1.249,NaN,1.389,NaN
3,4369,01/01/2022,1.359,1.359,1.479,1.599
4,5195,01/01/2022,1.409,1.409,1.509,1.619


In [6]:
df_gasolineras.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_estacion  42 non-null     int64  
 1   direccion    42 non-null     object 
 2   cod_postal   42 non-null     int64  
 3   latitud      42 non-null     float64
 4   longitud     42 non-null     float64
 5   rotulo       42 non-null     object 
 6   horario      42 non-null     object 
 7   municipio    42 non-null     object 
dtypes: float64(2), int64(2), object(4)
memory usage: 2.8+ KB


In [7]:
df_precios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47089 entries, 0 to 47088
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id_estacion             47089 non-null  int64  
 1   fecha                   47089 non-null  object 
 2   precio_gasoleo_a        46004 non-null  float64
 3   precio_gasoleo_premium  32054 non-null  float64
 4   precio_gasolina_95      43818 non-null  float64
 5   precio_gasolina_98      27543 non-null  float64
dtypes: float64(4), int64(1), object(1)
memory usage: 2.2+ MB


In [8]:
df_precios.describe()

,id_estacion,precio_gasoleo_a,precio_gasoleo_premium,precio_gasolina_95,precio_gasolina_98
count,47089.000000,46004.000000,32054.000000,43818.000000,27543.000000
mean,8874.081781,1.615661,1.637473,1.668314,1.824796
std,4164.742397,0.219800,0.200046,0.169220,0.165441
min,4369.000000,1.149000,1.149000,1.259000,1.379000
25%,5129.000000,1.455000,1.489000,1.559000,1.719000
50%,5318.000000,1.569000,1.589000,1.634000,1.789000
75%,13369.000000,1.759000,1.759000,1.739000,1.891000
max,16011.000000,2.259000,2.179000,2.299000,2.379000


## Análisis Exploratorio

In [9]:
df_gasolineras['id_estacion'].nunique() # Total de gasolineras en el municipio

42

In [10]:
df_gasolineras['cod_postal'].value_counts()

cod_postal
2007    15
2006    11
2005     5
2080     4
2001     3
2002     1
2099     1
2004     1
2328     1
Name: count, dtype: int64

Donde más gasolineras se ubican es en el **polígono Industrial de Campollano** (código postal `02007`) seguido de la área Oeste de la ciudad (`02006`)

In [11]:
import folium

coord_ab = [39.00, -1.86] # coordenadas orientativas

# Crea el mapa centrado en la ciudad
mapa_ab = folium.Map(location=coord_ab, zoom_start=12)

# Añade marcadores para cada gasolinera
for index, row in df_gasolineras.iterrows():
    folium.Marker(
        location=[row['latitud'], row['longitud']],
        popup=row['direccion'],
        tooltip=row['rotulo']
    ).add_to(mapa_ab)

mapa_ab

In [12]:
df_gasolineras['rotulo'].value_counts().head()

rotulo
REPSOL       9
CEPSA        7
INPEALSA     4
PLENOIL      2
CARREFOUR    1
Name: count, dtype: int64

Las gasolineras que predominan son las de **Repsol** y **Cepsa**

In [13]:
df_gasolineras['horario'].value_counts().head()

horario
L-D: 24H                              15
L-D: 06:00-22:00                       6
L-D: 07:00-23:00                       4
L-V: 07:00-21:00; S-D: 09:00-14:00     2
L-D: 06:30-22:30                       2
Name: count, dtype: int64

Hay un número significativo de gasolineras que abren 24 horas del día.

**NOTA** Para un mejor análisis, estaría bien extraer el tiempo total de apertura y no "fiarnos" de los strings

In [14]:
# Para un mejor manejo de las fechas
df_precios['fecha'] = pd.to_datetime(df_precios['fecha'],format='%d/%m/%Y')

fecha_minima = df_precios['fecha'].min()
fecha_maxima = df_precios['fecha'].max()

print(f"Fecha INICIO {fecha_minima} y Fecha FIN {fecha_maxima}")

Fecha INICIO 2022-01-01 00:00:00 y Fecha FIN 2024-12-31 00:00:00


In [15]:
df_precios['fecha'].nunique() # debería ser igual a 365  * 3 años (2022, 2023 y 2024) ~ 1095

1093

In [16]:
df_precios.isna().sum() / df_precios.shape[0] * 100

id_estacion                0.000000
fecha                      0.000000
precio_gasoleo_a           2.304147
precio_gasoleo_premium    31.928901
precio_gasolina_95         6.946421
precio_gasolina_98        41.508633
dtype: float64

Hay un gran número de valores perdidos para ciertos precios, veamos si siguen algun patrón.

Una primera razón puede ser que no todas las gasolineras oferten los mismo productos, así que empecemos por ahí

In [17]:
(
    df_precios
    .groupby(by=['id_estacion'])
    .agg(
        {
            'precio_gasoleo_a': 'count',
            'precio_gasoleo_premium': 'count',
            'precio_gasolina_95': 'count',
            'precio_gasolina_98': 'count'
        })
    .sort_values(by="precio_gasoleo_a", ascending=False)
    .head()
)

,precio_gasoleo_a,precio_gasoleo_premium,precio_gasolina_95,precio_gasolina_98
id_estacion,,,,
4369,1093,1093,1093,1093
5245,1093,1093,1093,1093
14377,1093,0,0,0
14287,1093,1093,1093,1093
13933,1093,0,1093,0


In [18]:
(
    df_precios
    .groupby(by=['id_estacion'])
    .agg(
        {
            'precio_gasoleo_a': 'count',
            'precio_gasoleo_premium': 'count',
            'precio_gasolina_95': 'count',
            'precio_gasolina_98': 'count'
        })
    .sort_values(by="precio_gasoleo_a", ascending=False)
    .tail()
)

,precio_gasoleo_a,precio_gasoleo_premium,precio_gasolina_95,precio_gasolina_98
id_estacion,,,,
5318,906,0,906,0
4394,895,895,895,0
13620,664,664,664,664
16011,308,0,308,0
14612,0,0,0,0


Al agrupar vemos el patrón que siguen los valores nulos:
- Si no hay ningun registro, significa que se producto no se vende en dicha gasolinera
- Si hay menos de `1093` registros (total de días en tres años), significa que ese producto se incorporó con el tiempo o bien que la gasolinera es nueva


**NOTA** Curioso caso de la gasolinera `14612` que no tiene registro de ningún tipo, quizás solo vende gas u otros combustibles

In [19]:
df_precios.duplicated().sum() / df_precios.shape[0] * 100

0.0

No hay duplicados, por tanto tenemos que ver como manejar los valores nulos

Aquí tenemos que tener en cuenta que hay varias `time-series`: una por cada combinación de gasolinera y producto. Es decir un total de 42 gasolineras X 4 productos = 168 líneas temporales a priori independientes.

Para ver la evolución de precios, tomemos una gasolinera como ejemplo para simplificar los gráficos

In [20]:
id_gasolinera = 4369 # se puede hacer al azar, la tomamos de la anterior consulta
nombre_gasolinera = df_gasolineras[df_gasolineras['id_estacion'] == id_gasolinera]['rotulo'].values[0]
df = df_precios[df_precios['id_estacion'] == id_gasolinera]

fig = px.line(
    df,
    x='fecha',
    y=['precio_gasoleo_a', 'precio_gasoleo_premium', 'precio_gasolina_95', 'precio_gasolina_98'], 
    title=f'Evolución de Precios Gasolinera {nombre_gasolinera}',
    labels={'value': 'Precio (€)', 'fecha': 'Fecha'}
)
fig.show()

Notas a tener en cuenta:
- Llama mucho la atención que el precio del gasóleo A y el Premium se solapa completamente
- Hubo un pico en el 2022 debido al inicio de la guerra en Ucrania
- Las tendencias de los distintos productos es bastante similar a lo largo del tiempo

In [21]:
 # Gasolineras más baratas
(
    df_precios
    # .assign(year=df_precios['fecha'].dt.year)
    .groupby(['id_estacion'])
    .agg(
        precio_medio_gasoleo_a=('precio_gasoleo_a', 'mean'),
        std_precio_gasoleo_a=('precio_gasoleo_a', 'std'),
        precio_medio_gasolina_95=('precio_gasolina_95', 'mean'),
        std_precio_gasolina_95=('precio_gasolina_95', 'std')
    )
    .reset_index()
    .merge(df_gasolineras[['id_estacion','rotulo', 'direccion']], on='id_estacion', how='left') 
    .sort_values(by=['precio_medio_gasoleo_a','precio_medio_gasolina_95'], ascending=True) # mas BARATAS!!!
    .head()
)

,id_estacion,precio_medio_gasoleo_a,std_precio_gasoleo_a,precio_medio_gasolina_95,std_precio_gasolina_95,rotulo,direccion
44,16011,1.301893,0.063434,1.423640,0.069622,PLENOIL,"CALLE CONSTANTINO ROMERO, S/N"
32,13369,1.484640,0.207859,1.532735,0.155014,GMOIL,"AVENIDA PRIMERA, S/N"
36,13933,1.497918,0.211721,1.542608,0.153613,PLENOIL,"CALLE FEDERICO GARCIA LORCA, 1"
6,4428,1.499046,0.217244,1.546328,0.161984,FAMILY ENERGY,"CALLE ALCALDE CONANGLA (C.C. EROSKI), S/N"
42,15000,1.507103,0.218641,1.584357,0.160104,A&A,"AVENIDA ESCRITOR RODRIGO RUBIO, 3"


In [22]:
 # Gasolineras más caras
(
    df_precios
    # .assign(year=df_precios['fecha'].dt.year)
    .groupby(['id_estacion'])
    .agg(
        precio_medio_gasoleo_a=('precio_gasoleo_a', 'mean'),
        std_precio_gasoleo_a=('precio_gasoleo_a', 'std'),
        precio_medio_gasolina_95=('precio_gasolina_95', 'mean'),
        std_precio_gasolina_95=('precio_gasolina_95', 'std')
    )
    .reset_index()
    .merge(df_gasolineras[['id_estacion','rotulo', 'direccion']], on='id_estacion', how='left') 
    .sort_values(by=['precio_medio_gasoleo_a','precio_medio_gasolina_95'] , ascending=False) # mas CARAS!!!
    .head()
)

,id_estacion,precio_medio_gasoleo_a,std_precio_gasoleo_a,precio_medio_gasolina_95,std_precio_gasolina_95,rotulo,direccion
2,4413,1.913080,0.417415,1.988298,0.367743,ABOIL,"POLIGONO CAMPOLLANO AVENIDA -0-, 69"
33,13620,1.733367,0.204405,1.744949,0.146962,CEPSA,"POLIGONO CAMPOLLANO, 55"
26,11048,1.686984,0.180787,1.719813,0.126689,CEPSA,CARRETERA N-322 KM. 349
7,4790,1.686823,0.184423,1.721681,0.138784,"INLOCOR S.L. ""CEPSA""","CARRETERA C-M 332 KM. 2,6"
37,14123,1.674496,0.177693,1.710880,0.134343,CEPSA,"PASEO CUBA (LA), 36"
